In [1]:
initial_prompt = (
        "You are a sentiment analysis classifier. Determine whether the provided text expresses a positive sentiment. "
        "Respond with '1' if it is positive, or '0' if it is negative."
    )

# Output format prompt
output_format_prompt = (
    "You are to act as a binary responder. For every question asked, reply strictly with '1' for positive or '0' for negative. "
    "Do NOT include any additional text or explanation."
)

In [2]:
output_schema = {
    'key_to_extract': None,  # Set to None for direct output
    'value_mapping': None,   # Set to None for direct mapping
    'regex_pattern': r'^(0|1)$',  # Match the entire output for binary classification
    'use_json_mode': False
}

In [3]:
# Set number of optimization iterations
iterations = 10

In [4]:
# Define model providers and models for evaluation and optimization
eval_provider = "ollama"
eval_model = "llama3.1"
optim_provider = "ollama"
optim_model = "llama3.1"

In [5]:
# Path to the CSV file containing review data for evaluation
eval_datapath = "sentiments.csv"
sample_size = 20

------------------------------------------------------------------------------------------

In [6]:
# Import necessary libraries
import pandas as pd
import sys
import os
# Add the parent directory to sys.path
# Use getcwd() to get the current working directory for Jupyter notebooks
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from src.iterative_prompt_optimization import optimize_prompt

In [7]:
!python -m src.iterative_prompt_optimization.dashboard_generator

/opt/anaconda3/bin/python: Error while finding module specification for 'src.iterative_prompt_optimization.dashboard_generator' (ModuleNotFoundError: No module named 'src')


In [8]:
# Load and prepare data
eval_data = pd.read_csv(eval_datapath, encoding='ISO-8859-1', usecols=['Text', 'Sentiment'])
eval_data.columns = ['text', 'label']
# Randomly select 50 positive and 50 negative samples
eval_data = (
    eval_data.groupby('label')
    .apply(lambda x: x.sample(n=round(sample_size/2), random_state=42))
    .reset_index(drop=True)
)
# Shuffle the DataFrame randomly
eval_data = eval_data.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Evaluation data shape: {eval_data.shape}")
print(eval_data.head())

Evaluation data shape: (20, 2)
                                                text  label
0  I hate guns and have never murdered anyone, bu...      0
1  "A Cry in the Dark" is a masterful piece of ci...      1
2  I was watching the Perfect Storm, and thought ...      1
3  If you ask me the first one was really better ...      0
4  The movie 'Heart of Darkness', based on the 18...      0


In [9]:
# best_prompt, best_metrics = optimize_prompt(
#     initial_prompt,
#     output_format_prompt,
#     eval_data,
#     iterations)

In [10]:
# Run the prompt optimization process
best_prompt, best_metrics = optimize_prompt(
    initial_prompt,
    output_format_prompt,
    eval_data,
    iterations,
    eval_provider=eval_provider,
    eval_model=eval_model,
    optim_provider=optim_provider,
    optim_model=optim_model,
    output_schema=output_schema,
    use_cache=True  # Set to False if you want to disable caching
)
# After running the optimization process, you can analyze the results by checking 
# the generated log files in the `runs/prompt_optimization_logs_YYYYMMDD_HHMMSS` directory.

Selected evaluation provider: ollama
Selected evaluation model: llama3.1
Selected optimization provider: ollama
Selected optimization model: llama3.1
Estimated token usage: 1071600
Estimated cost: $0 API Costs - Running on Local Hardware

Do you want to proceed with the optimization? (Y/N): 
Iteration 1/10


╭────────────────────────────────────────────── Current Full Prompt ──────────────────────────────────────────────╮
│ You are a sentiment analysis classifier. Determine whether the provided text expresses a positive sentiment.    │
│ Respond with '1' if it is positive, or '0' if it is negative.                                                   │
│                                                                                                                 │
│ You are to act as a binary responder. For every question asked, reply strictly with '1' for positive or '0' for │
│ negative. Do NOT include any additional text or explanation.                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Processing text 1/20
Prediction 1/20: 1 | Ground Truth: 0 ❌ (FP)
Processing text 2/20
Prediction 2/20: 1 | Ground Truth: 1 ✅ (TP)
Processing text 3/20
Prediction 3/20: 1 | Ground Truth: 1 ✅ (TP)
Processing text 4/20
Prediction 4/20: 0 | Ground Truth: 0 ✅ (TN)
Processing text 5/20
Prediction 5/20: 0 | Ground Truth: 0 ✅ (TN)
Processing text 6/20
Prediction 6/20: 0 | Ground Truth: 0 ✅ (TN)
Processing text 7/20
Prediction 7/20: 1 | Ground Truth: 1 ✅ (TP)
Processing text 8/20
Prediction 8/20: 0 | Ground Truth: 0 ✅ (TN)
Processing text 9/20
Prediction 9/20: 1 | Ground Truth: 1 ✅ (TP)
Processing text 10/20
Prediction 10/20: 1 | Ground Truth: 1 ✅ (TP)
Processing text 11/20
Prediction 11/20: 1 | Ground Truth: 1 ✅ (TP)
Processing text 12/20
Prediction 12/20: 1 | Ground Truth: 0 ❌ (FP)
Processing text 13/20
Prediction 13/20: 0 | Ground Truth: 0 ✅ (TN)
Processing text 14/20
Prediction 14/20: 1 | Ground Truth: 1 ✅ (TP)
Processing text 15/20
Prediction 15/20: 0 | Ground Truth: 0 ✅ (TN)
Processing te

Evaluation Metrics - Iteration 1
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Metric              ┃  Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Precision           │ 0.7692 │
│ Recall              │ 1.0000 │
│ Accuracy            │ 0.8500 │
│ F1-score            │ 0.8696 │
│ Valid Predictions   │     20 │
│ Invalid Predictions │      0 │
└─────────────────────┴────────┘


Analyzing misclassifications...


...


Analyzing misclassifications...


╭──────────────────────────────────────── Analysis of Misclassifications ─────────────────────────────────────────╮
│                                                                                                                 │
│ After analyzing the misclassifications, I've identified specific examples where the model missed subtle         │
│ indicators of positive cases and strategies to improve the classification prompt to reduce false negatives.     │
│                                                                                                                 │
│ **False Negatives (Positive texts incorrectly classified as negative):**                                        │
│                                                                                                                 │
│ 1. **Lack of explicit negativity**: In the first text, the reviewer explicitly states that the movie is "one    │
│ with little to no detail" and has a "schizophrenic like plot line," which might lead the model to assume it's a │
│ negative review. However, upon closer inspection, the reviewer highlights the importance of imagination in the  │
│ book and compares it favorably to the movie. The phrase "If you have read the book then you know that little to │
│ none of the important 'story making' scenes were put into the movie" suggests that the reviewer is focusing on  │
│ the positives of the book, implying a positive tone.                                                            │
│ 2. **Irony and sarcasm**: In the second text, the reviewer describes the movie as "one of the cheapest action   │
│ flicks of the eighties" but frames it as a positive aspect, saying it's perfect for those who enjoy great       │
│ trash. The use of irony and sarcasm might have led the model to misinterpret the tone.                          │
│                                                                                                                 │
│ **Strategies to Improve Recall:**                                                                               │
│                                                                                                                 │
│ 1. **Be more sensitive to nuanced positivity**: Consider adding phrases or keywords that indicate subtle        │
│ positivity, such as "enjoyable despite," "a guilty pleasure," or "a fun watch." This would help the model       │
│ recognize positive cases that were missed.                                                                      │
│ 2. **Emphasize contextual understanding**: Update the prompt to emphasize the importance of considering context │
│ when evaluating tone and language. This could involve adding phrases like "Consider the reviewer's intent" or   │
│ "Take into account the tone and language used."                                                                 │
│ 3. **Reduce emphasis on explicit criticism**: Modify the prompt to prioritize identifying positive reviews,     │
│ even if they contain criticisms or mitigating factors. This would encourage the model to focus more on finding  │
│ positives rather than dwelling on negatives.                                                                    │
│ 4. **Introduce a threshold for positivity**: Implement a minimum threshold for implicit positivity before       │
│ classifying a review as positive.                                                                               │
│                                                                                                                 │
│ **Specific Examples:**                                                                                          │
│                                                                                                                 │
│ 1. Add phrases like "even though" or "despite" to indicate that the reviewer is focusing on the positives of    │
│ the text.                                             


Generating new prompt...

Iteration 8/10


╭────────────────────────────────────────────── Current Full Prompt ──────────────────────────────────────────────╮
│ Based on the analysis and strategies provided, I've refined the prompt to improve recall while maintaining a    │
│ high level of precision and accuracy.                                                                           │
│                                                                                                                 │
│ **Improved Prompt:**                                                                                            │
│                                                                                                                 │
│ "I am a sentiment analysis classifier trained to detect subtle nuances in language, with a focus on positivity  │
│ and contextual understanding. My primary goal is to accurately classify texts as '1' (positive) or '0'          │
│ (negative), while being sensitive to tone, irony, sarcasm, humor, and subtle indicators of enjoyment or         │
│ appreciation.                                                                                                   │
│                                                                                                                 │
│ To improve my recall and accuracy, I will consider the following:                                               │
│                                                                                                                 │
│ * Nuanced expressions of positive sentiment, such as "fairly good," "not bad," or "pretty cool."                │
│ * Implicit positivity, including implied agreement or approval, even when not explicitly stated.                │
│ * Subtle indicators of enjoyment or appreciation, like emphatic language ("love," "adore"), enthusiastic        │
│ phrases ("can't wait," "excited"), or playful teasing.                                                          │
│                                                                                                                 │
│ When classifying texts that employ complex language or implicit positivity, I will consider:                    │
│                                                                                                                 │
│ * The presence of redeeming qualities or mitigating factors mentioned by the reviewer.                          │
│ * The context in which criticisms are presented, such as whether they are framed as humorous or ironic.         │
│ * The tone and language used to convey the negative comments, including any signs of sarcasm or exaggeration.   │
│                                                                                                                 │
│ When encountering reviews that criticize aspects but still express overall positive sentiment, I will consider: │
│                                                                                                                 │
│ * Whether the reviewer's criticism is done in a way that suggests they're enjoying the absurdity rather than    │
│ genuinely hating the content.                                                                                   │
│ * The presence of phrases like "mindless fun," "a nice ," or "unintentional humour" implies a positive tone.    │
│                                                                                                                 │
│ To accurately classify texts that employ irony or sarcasm, I will be aware of cues like "just kidding" or "no   │
│ offense." In situations where the text's tone is ambiguous or open to interpretation, I will strive to make an  │
│ accurate classification based on contextual understanding and linguistic features.                              │
│                                                                                                                 │
│ When classifying reviews with sarcastic or ironic tone

Processing text 1/20
Prediction 1/20: 1 | Ground Truth: 0 ❌ (FP)
Processing text 2/20
